In [2]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

In [3]:
import requests
import pandas as pd
url = 'https://www.boligsiden.dk/propertyresult/getdata?searchId=b4fae611b9444fa88f6dd6cd5f39de8d&pageNumber=1&sortKey=12&sortDescending=false&displayTab=1&itemsPerPage=60'
response = requests.get(url)
response.status_code

200

In [4]:
response.json().keys()

dict_keys(['newSearchId', 'result', 'search', 'mapTokens'])

In [5]:
page = response.json()['result']

In [6]:
boligdf = pd.DataFrame(page['properties'])

In [7]:
boligdf = pd.DataFrame(page['properties'])

In [8]:
pd.options.display.max_columns = 100

In [9]:
totalcount = page['totalCount']
totalcount

64339

In [10]:
itemsperpage = 60

In [11]:
pages = round(totalcount/itemsperpage, 0)
pages

1072.0

In [12]:
def url_constructor():
    urls = []
    part_one = 'https://www.boligsiden.dk/propertyresult/getdata?searchId=b4fae611b9444fa88f6dd6cd5f39de8d&pageNumber='
    part_two = '&sortKey=12&sortDescending=false&displayTab=1&itemsPerPage=60'
    for i in range(1, int(pages)+1):
        urls.append(part_one + str(i) + part_two)
    return urls
        #print(i)

tests = url_constructor()

In [13]:
len(tests)

1072

In [14]:
import time
import random
import json
import tqdm

def scrape_append(url_list):
    bolig_df = pd.DataFrame()
    
    for url_link in tqdm.tqdm(url_list):
        time.sleep(0.5)
        response, call_id = connector.get(url_link,'DataScience_Boligsiden')
        Y = response.text
        Y_json = json.loads(Y)
        #data = Y_json['JobPositionPostings']
        pagedata = Y_json['result']
        data = pagedata['properties']
        df = pd.DataFrame(data)
        bolig_df = pd.concat([df, bolig_df], ignore_index=True)
    
    return bolig_df
url_input = url_constructor()
url_in_test = url_input[0:20]
housing_df = scrape_append(url_input)

100%|██████████| 1072/1072 [29:07<00:00,  1.69s/it]


In [15]:
housing_df.shape

(64317, 39)

In [155]:
def hnum(addresses):
    street_list=[]
    regex = re.compile('\d+') 
    for address in addresses:
        street= regex.split(address,maxsplit=1)
        street_list.append(street)
    return street_list

street=hnum(housing_df["address"])
street

[['Skovhaven ', ''],
 ['Planetvej ', ''],
 ['Thorsvænge ', ''],
 ['Thorsvænge ', ''],
 ['Thorsvænge ', ''],
 ['Højbogårdsvej ', ''],
 ['Morten Nielsensvej ', ''],
 ['Torebyvej ', ''],
 ['Torupvej ', ''],
 ['Strandparken ', ''],
 ['Matildesvej, Prisgruppe ', ''],
 ['Matildesvej, Prisgruppe ', ''],
 ['Rasmus Larsens Vænge ', ''],
 ['Rasmus Larsens Vænge ', ''],
 ['Rasmus Larsens Vænge ', ''],
 ['Sletten ', ''],
 ['Bredgade ', ''],
 ['Vester Storetoft ', 'A'],
 ['K. H. Kofoedsvej ', ''],
 ['Møgelbjerg'],
 ['Rolighed ', ''],
 ['Røgindvej ', ''],
 ['Thorsvænge ', ''],
 ['Gammel Færgevej ', ''],
 ['Tranevej ', 'C'],
 ['Tjørnemose ', ''],
 ['Madsbadvej ', ''],
 ['Muffparken ', ''],
 ['Ingeborgvej ', ''],
 ['Egernvænget ', ''],
 ['Hunderup Klitvej ', ''],
 ['Hunderup Klitvej ', ''],
 ['Hunderup Klitvej ', ''],
 ['Ulvevej ', ''],
 ['Skjernvej ', ''],
 ['Eskovvej ', ''],
 ['Skolegade ', ''],
 ['SØHOLM PARKVEJ ', ''],
 ['Alkajavej ', ''],
 ['Ribevej ', ''],
 ['Præstevænget ', ''],
 ['Duevænget ',

In [236]:
housing_df["address"].str.extract("^([A-Za-zæøåÆØÅ. ]+) (\d+)")
housing_df["street"]=housing_df["address"].str.extract('^([5A-Za-zæøåÆØÅ. "]+)')
housing_df["number"]=housing_df["address"].str.extract("(\d+.*|\d+)")
housing_df[housing_df["number"].isnull()] # 37 uden vejnumre - er byggegrunde
housing_df[housing_df["street"].isnull()] # 18 vejnavne, der har mærkelige symboler

,address,agentChainName,agentsLogoLink,areaParcel,areaPaymentCash,areaResidential,areaWeighted,canShowSalesPeriodTotal,city,dateAnnounced,dateRemoved,downPayment,energyMark,energyMarkLink,financing,hasOpenHouse,id,imageLink300X200,isArchive,isFavorite,itemType,itemTypeName,nextOpenHouse,nextOpenHouseSignup,numberOfRooms,openHouseRedirectLink,paymentCash,paymentExpenses,paymentGross,paymentNet,placeName,postal,priceDevelopment,propertyLink,rating,redirectLink,salesPeriod,salesPeriodTotal,uniqueNumber,street,number
4322,?ster Hassing Vej 172,KEC Bolig,https://pic.boligsiden.dk/agent/074dc5391fee4b...,8.801,6.376,101,109,True,Gandrup,03-11-2016,,35.000,,https://sparenergi.dk/forbruger/vaerktoejer/de...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,2d12d1c3a7cc4fcd96f43f67e4413563,https://pic.boligsiden.dk/property/300x200/1/2...,False,False,600,Landejendom,01. jan. 00:00,False,3,None,695.000,2.313,2.772,2.463,,9362,-30%,~/salg/319086895,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/2d12d...,1.022,-,319086895,NaN,172
12829,?struplundvej 34,Linda Riis Ejendomsmægler,https://pic.boligsiden.dk/agent/68533efff93e44...,1.399,10.877,183,211,True,Brædstrup,24-05-2018,,115.000,c,https://sparenergi.dk/forbruger/vaerktoejer/fi...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,543c0ee37abf4e33ab86ec052845309b,https://pic.boligsiden.dk/property/300x200/1/5...,False,False,100,Villa,01. jan. 00:00,False,6,None,2.295.000,2.338,9.021,8.045,,8740,0%,~/salg/228634123,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/543c0...,455,-,228634123,NaN,34
14007,?sterskovvej 49,Ejendomsmæglerfirmaet Lilienhoff A/S,https://pic.boligsiden.dk/agent/4fd0a4dd45b941...,123.318,17.857,178,196,True,Kruså,18-06-2018,,175.000,d,https://sparenergi.dk/forbruger/vaerktoejer/fi...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,abdefa1b2b09454f993a6b27f83a8536,https://pic.boligsiden.dk/property/300x200/1/a...,False,False,600,Landejendom,01. jan. 00:00,False,4,None,3.500.000,1.754,13.695,12.186,,6340,0%,~/salg/312891622,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/abdef...,430,-,312891622,NaN,49
14473,?dalen 4,Linda Riis Ejendomsmægler,https://pic.boligsiden.dk/agent/68533efff93e44...,583,8.945,118,128,True,Skanderborg,28-06-2018,,60.000,g,https://sparenergi.dk/forbruger/vaerktoejer/fi...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,99ecea93f1e24fd786cad381d08046a0,https://pic.boligsiden.dk/property/300x200/1/9...,False,False,100,Villa,01. jan. 00:00,False,3,None,1.145.000,1.212,4.518,4.028,Vrold,8660,-12%,~/salg/173402824,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/99ece...,420,694,173402824,NaN,4
14498,?. H?gildvej 4A,Lilienhoff Østjylland,https://pic.boligsiden.dk/agent/393100a53d574d...,309.117,10.830,745,572,True,Herning,27-06-2018,,310.000,c,https://sparenergi.dk/forbruger/vaerktoejer/fi...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,b1e2c4df4e5e4471b7f835d40a051695,https://pic.boligsiden.dk/property/300x200/1/b...,False,False,600,Landejendom,01. jan. 00:00,False,15,None,6.195.000,4.772,24.912,21.817,,7400,-8%,~/salg/498865646,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/b1e2c...,421,1.055,498865646,NaN,4A
16549,?sters?vej 11,Hedegaard Madsen A/S,https://pic.boligsiden.dk/agent/a13dfc24a07e45...,1.457,12.871,153,155,True,Hjørring,20-08-2018,,100.000,,https://sparenergi.dk/forbruger/vaerktoejer/de...,{'link': 'https://raadgivning.boligsiden.dk/bo...,False,94e43fb767b64ce59cfa34106ee5d02a,https://pic.boligsiden.dk/property/300x200/1/9...,False,False,400,Fritidsbolig,01. jan. 00:00,False,6,None,1.995.000,2.402,8.150,7.151,,9800,0%,~/salg/574067552,"{'ratings': {'conditionRating': None, 'kitchen...",https://www.boligsiden.dk/viderestilling/94e43...,367,-,574067552,NaN,11
29250,"?rumvej 35, 

In [268]:
housing_df['DKK_Development']=housing_df["priceDevelopment"].str.extract("(\d+|-\d+)")
# housing_df['DKK_Development'].astype(int)
# pd.to_numeric.(housing_df['DKK_Development'])
housing_df['DKK_Development']

SyntaxError: invalid syntax (<ipython-input-268-eaf5d9cf3af5>, line 3)

In [16]:
housing_df.to_csv('housing_data2.csv')

In [244]:
pd.set_option('display.max_rows', 250)
housing_df["agentChainName"].value_counts().to_frame()

,agentChainName
EDC,13539
Nybolig,11644
home,9185
danbolig,8236
RealMæglerne,3228
Estate,2911
LokalBolig,1431
Ejendomsmæglerfirmaet John Frandsen,401
BoligOne,337
Mikkelsens Ejendomskontor,304


In [18]:
housing_df['rating'] = housing_df['rating'].astype(str)
housing_df['financing'] = housing_df['financing'].astype(str)
housing_distinct = housing_df.drop_duplicates()
housing_distinct.shape

(64316, 39)

In [19]:
#housing_distinct = housing_df.drop_duplicates()
#housing_distinct.shape
housing_df.dtypes

address                    object
agentChainName             object
agentsLogoLink             object
areaParcel                 object
areaPaymentCash            object
areaResidential            object
areaWeighted               object
canShowSalesPeriodTotal      bool
city                       object
dateAnnounced              object
dateRemoved                object
downPayment                object
energyMark                 object
energyMarkLink             object
financing                  object
hasOpenHouse                 bool
id                         object
imageLink300X200           object
isArchive                    bool
isFavorite                   bool
itemType                    int64
itemTypeName               object
nextOpenHouse              object
nextOpenHouseSignup          bool
numberOfRooms              object
openHouseRedirectLink      object
paymentCash                object
paymentExpenses            object
paymentGross               object
paymentNet    

In [20]:
housing_distinct.to_csv('housing_data_noduplicates.csv')

In [ ]:

url="https://dawa.aws.dk/adresser"

def dawa_api_searcher(adress):
    